# 🏠 DeepRoof-2026: Master Training Lab

### 🛠 Step 1: Intelligent Environment Initialization
This cell handles **Dynamic Library Matching**, venv activation, and CUDA linking.

In [ ]:
import os
import sys
import subprocess
import torch
import ctypes
import glob
from pathlib import Path

print("🛰 Initializing DeepRepair Protocol V3...")

# --- 1. PATH RESOLUTION ---
project_root = Path("/workspace/roof")
if not project_root.exists():
    project_root = Path(os.getcwd()).parent

venv_path = project_root / "venv"
if not venv_path.exists():
    venv_path = project_root / ".venv"

if venv_path.exists():
    site_pkgs = list(venv_path.glob("lib/python*/site-packages"))
    if site_pkgs:
        if str(site_pkgs[0]) not in sys.path:
            sys.path.insert(0, str(site_pkgs[0]))
        sys.executable = str(venv_path / "bin" / "python")
    print(f"🐍 Venv Active: {venv_path}")

if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

# --- 2. NUCLEAR CUDA LINKER (FIXES libcudart.so.11.0) ---
def nuclear_cuda_fix():
    if not torch.cuda.is_available():
        print("❄️ CPU Mode: Skipping CUDA linking.")
        return True

    print("🔍 Searching for libcudart.so.11.0...")
    
    # Deep search locations
    search_patterns = [
        "/usr/local/cuda*/lib64/libcudart.so.11.0",
        "/usr/lib/x86_64-linux-gnu/libcudart.so.11.0",
        str(venv_path / "lib/python*/site-packages/nvidia/cuda_runtime/lib/libcudart.so.11.0")
    ]
    
    found_lib = None
    for pattern in search_patterns:
        matches = glob.glob(pattern)
        if matches:
            found_lib = matches[0]
            break
            
    if not found_lib:
        print("📦 Missing libcudart.so.11.0. Installing nvidia-cuda-runtime-cu11...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", "nvidia-cuda-runtime-cu11"])
        # Re-search after install
        matches = glob.glob(str(venv_path / "lib/python*/site-packages/nvidia/cuda_runtime/lib/libcudart.so.11.0"))
        if matches: found_lib = matches[0]
        else: 
            print("❌ ERROR: Still could not find libcudart after install.")
            return False

    print(f"📍 Found linking target: {found_lib}")
    try:
        ctypes.CDLL(found_lib, mode=ctypes.RTLD_GLOBAL)
        print("✅ CUDA Runtime successfully force-loaded into memory.")
    except Exception as e:
        print(f"⚠️ Force-load failed: {e}")
        return False
    
    return True

# --- 3. DYNAMIC MMCV VERSION MATCHER ---
def match_mmcv_to_torch():
    torch_ver = torch.__version__
    cuda_ver = torch.version.cuda
    print(f"🔍 Detected: Torch {torch_ver} | CUDA {cuda_ver}")
    
    # Attempt verify
    try:
        import mmcv
        from mmcv.ops import point_sample
        print("✅ MMCV Binary matches Torch version.")
        return True
    except (ImportError, ModuleNotFoundError) as e:
        print(f"❌ Mismatch detected: {e}")
        print("🔄 Resolving correct MMCV version...")
    
    # Uninstall bad version
    subprocess.run([sys.executable, "-m", "pip", "uninstall", "-y", "mmcv"], check=False)
    
    # Construct URL based on Torch version
    base_url = "https://download.openmmlab.com/mmcv/dist"
    if "2.1" in torch_ver:
        target_url = f"{base_url}/cu118/torch2.1/index.html"
    elif "2.0" in torch_ver:
        target_url = f"{base_url}/cu118/torch2.0/index.html"
    elif "1.13" in torch_ver:
        target_url = f"{base_url}/cu117/torch1.13/index.html"
    else:
        print("⚠️ Unknown Torch version. Trying generic 'mmcv>=2.0.0' install via MIM...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-U", "openmim"])
        subprocess.check_call([sys.executable, "-m", "mim", "install", "mmcv>=2.0.0"])
        return False
        
    print(f"⬇️ Installing from: {target_url}")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "mmcv==2.2.0", "-f", target_url])
    return False

# --- 4. ASSERTION PATCH ---
def patch_assertions():
    try:
        # Use glob to find mmseg even if import fails
        matches = glob.glob(str(venv_path / "lib/python*/site-packages/mmseg/__init__.py"))
        if matches:
            with open(matches[0], 'r') as f: content = f.read()
            if "assert (mmcv_min_version" in content:
                print("🔥 Nuking assertions...")
                # (Simplified overwrite for brevity)
                subprocess.run(["sed", "-i", "/assert (mmcv_min_version/d", matches[0]])
                subprocess.run(["sed", "-i", "/Please install mmcv/d", matches[0]])
                return False
    except: pass
    return True

if nuclear_cuda_fix() and match_mmcv_to_torch() and patch_assertions():
    print(f"🚀 System Live | {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'CPU'}")
else:
    print("\n⚠️  ENVIRONMENT UPDATED. PLEASE RESTART KERNEL.")

## 📂 1. Dataset Preview

Visualize the **OmniCity** satellite imagery and ground truth **Masks** + **Surface Normals**.

In [ ]:
def preview_dataset(num_samples=3):
    import matplotlib.pyplot as plt
    import numpy as np
    import cv2
    
    data_path = project_root / "data/OmniCity"
    train_file = data_path / 'train.txt'
    
    if not train_file.exists():
        print(f"❌ Multi-task training data not found at {data_path}.")
        return
        
    with open(train_file, 'r') as f:
        sample_ids = [line.strip() for line in f.readlines()[:num_samples]]
    
    fig, axes = plt.subplots(num_samples, 3, figsize=(15, 5 * num_samples))
    for i, sid in enumerate(sample_ids):
        img = cv2.cvtColor(cv2.imread(str(data_path / 'images' / (sid + '.jpg'))), cv2.COLOR_BGR2RGB)
        
        mask = cv2.imread(str(data_path / 'masks' / (sid + '.png')), cv2.IMREAD_UNCHANGED)
        mask_vis = cv2.applyColorMap(((mask % 20) * 12).astype(np.uint8), cv2.COLORMAP_JET)
        
        axes[i, 0].imshow(img); axes[i, 0].set_title(sid); axes[i, 0].axis('off')
        axes[i, 1].imshow(mask_vis); axes[i, 1].set_title("Mask"); axes[i, 1].axis('off')
        
        norm_path = data_path / 'normals' / (sid + '.npy')
        if norm_path.exists():
            normals = np.load(str(norm_path))
            axes[i, 2].imshow(((normals + 1) * 127.5).astype(np.uint8))
        axes[i, 2].set_title("Normals"); axes[i, 2].axis('off')
        
    plt.tight_layout(); plt.show()

preview_dataset(num_samples=2)

## ⚙️ 2. Scratch Training Configuration (Epoch-Based)

We are using the **MASTER EPOCH-BASED SCRATCH PROFILE**:
- **Duration**: 150 Epochs (~160k steps).
- **Val Interval**: Every 1 Epoch (Reports results per-epoch).
- **No Pre-Training**: `load_from = None`.
- **Checkpoints**: Interval snapshots every 5 epochs + `best_mIoU.pth`.

In [ ]:
from mmengine.config import Config

CONFIG_PATH = str(project_root / "configs/deeproof_scratch_swin_L.py")
WORK_DIR = str(project_root / "work_dirs/swin_l_scratch_v1")

cfg = Config.fromfile(CONFIG_PATH)
cfg.work_dir = WORK_DIR
cfg.data_root = str(project_root / "data/OmniCity/")
cfg.train_dataloader.dataset.data_root = cfg.data_root
cfg.val_dataloader.dataset.data_root = cfg.data_root

print(f"🏆 MASTER SCRATCH CONFIG LOADED")
print(f"🔥 Max Epochs: {cfg.train_cfg.max_epochs}")
print(f"📉 Initial LR: {cfg.optimizer.lr}")
print(f"📊 Reporting Interval: Every Epoch")

## 🚀 3. Kickoff Training

This will invoke the `mmengine.Runner` and begin the full model convergence process. **Detailed stats will print to this output at the end of every epoch.**

In [ ]:
import torch
from mmengine.runner import Runner

print(f"🚀 Starting Master Trainer on: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'CPU'}")

runner = Runner.from_cfg(cfg)
runner.train()

## 📊 4. Monitoring & Metrics

Run this cell during or after training to visualize performance trends.

In [ ]:
import json
import matplotlib.pyplot as plt

def plot_training_logs(log_path):
    if not os.path.exists(log_path):
        print("🕒 No logs found yet.")
        return
        
    iters, losses, miou = [], [], []
    with open(log_path, 'r') as f:
        for line in f:
            data = json.loads(line)
            if 'loss' in data:
                iters.append(data.get('iter', 0))
                losses.append(data['loss'])
            if 'mIoU' in data:
                miou.append(data['mIoU'])
                
    plt.figure(figsize=(10, 5))
    plt.plot(iters, losses, label='Loss')
    plt.title("Training Progress"); plt.show()

# log_json = glob.glob(os.path.join(WORK_DIR, "*/vis_data/scalars.json"))
# if log_json: plot_training_logs(log_json[-1])